# Programatic navigation

This notebook demonstrates programatic navigation of a Gosling visualization via Gos.

In [ ]:
#! pip install gosling[all]==0.0.11
import gosling as gos

We first create a simple bar visualization for a scATAC-seq "pseudobulk" track from [Corces et. al (Nature Genetics, 2020)](https://www.nature.com/articles/s41588-020-00721-x) multi-omic atlas of the human brain. This scATAC-seq track is stored in a separate BigWig file and represents the normalized aggregate signal for excitatory neuron cells.

In [ ]:
data = gos.bigwig(
    url="https://s3.amazonaws.com/gosling-lang.org/data/ExcitatoryNeurons-insertions_bin100_RIPnorm.bw",
    column="position",
    value="peak",
)
    
track = gos.Track(data, width=700).mark_bar().encode(
    x=gos.X("position:G"),
    y=gos.Y("peak:Q", axis="right"),
)

We can render the visualiztion in the notebook with the `track.view()` method. By default, the visualization is intialized to displays the _entire_ genome.

In [ ]:
track.view()

However, we are often interested in specific genomic _regions_, and navigating to those views manually can be cumbersome. Fortuately, the view location can be set using the `xDomain` via the Gosling grammar. 

With `gos`, we can create multiple _views_ of the same `track` easily.

In [ ]:
track.view(
    xDomain=gos.GenomicDomain(chromosome="chr13") # show an entire chromosome
)

In [ ]:
track.view(
    xDomain=gos.GenomicDomain(chromosome="chr13", interval=[31500000, 33150000]) # a specific interval
)

The Gosling grammar only defines the initial view location. This default behavior is useful for experimenting with visual encodings for a dataset, but there is limited control from Python of the resulting viewer.

An instance of `GoslingWidget` can be created for any Gos visualization by calling the `View.widget()` method. This returns a "live" viewer which can be interacted within Python.

In [ ]:
view = track.view(id="view0") # necessary to keep track of this view
widget = view.widget()
widget

We can now call the `widget.zoom_to()` API to navigate the viewer from Python!

In [ ]:
widget.zoom_to(view.id, "chr17") # zoom our view to a particular chromosome

Or combine our `widget` with other Jupyter widgets to create an interactive GUI.

In [ ]:
import ipywidgets 

# A dropdown to navigate the viewer to particular genomic regions
dropdown = ipywidgets.Dropdown(
    options=[
        ("TP53", "chr17:7668421-7687490"),
        ("TNF", "chr6:31575565-31578336"),
    ],
    description='Gene:',
)

dropdown.observe(lambda c: widget.zoom_to(view.id, c.new) if c.type == 'change' and c.name == 'value' else None)

ipywidgets.VBox([dropdown, widget])